# Dielectric interfaces

In [ ]:
import matplotlib.pyplot as plt
import meow as mw
import numpy as np
import tidy3d as td
from matplotlib import colors
from scipy.ndimage import convolve
from tidy3d.components.mode.solver import compute_modes as _compute_modes

mw.cache.disable_cache();

In [ ]:
w = 4.0
hc = 2.0
hs = 1.0
nsi = 3
nsl = 1.5
wl = 1.5
W, L, H = 12, 12, 12

In [ ]:
silicon = mw.SampledMaterial(
    name="si",
    params={"wl": [wl]},
    n=[nsi],
    meta={"color": (0.9, 0, 0, 0.9)},
)
silicon_oxide = mw.SampledMaterial(
    name="sio2",
    params={"wl": [wl]},
    n=[nsl],
)
core = mw.Structure(
    geometry=mw.Prism(
        axis="y",
        h_min=H // 3,
        h_max=H // 3 + hc,
        poly=[
            (-1, W // 2 - w / 2),
            (-1, W // 2 + w / 2),
            (L + 1, W // 2 + w / 2),
            (L + 1, W // 2 - w / 2),
        ],
    ),
    material=silicon,
)
slab = mw.Structure(
    geometry=mw.Prism(
        axis="y",
        h_min=H // 3,
        h_max=H // 3 + hs,
        poly=[(-1, -1), (-1, W + 1), (L + 1, W + 1), (L + 1, -1)],
    ),
    material=silicon,
)
box = mw.Structure(
    geometry=mw.Prism(
        axis="y",
        h_min=-1,
        h_max=H // 3,
        poly=[(-1, -1), (-1, W + 1), (L + 1, W + 1), (L + 1, -1)],
    ),
    material=silicon_oxide,
)

structs = [core, slab]  # , box]

cell1 = mw.Cell(
    structures=structs,
    mesh=mw.Mesh2D(
        x=np.linspace(0, W, 19),
        y=np.linspace(0, H, 19),
        num_pml=(0, 0),
        ez_interfaces=False,
    ),
    z_min=L / 2,
    z_max=L / 2,
)
cs1 = mw.CrossSection.from_cell(
    cell=cell1,
    env=mw.Environment(wl=wl),
)

mw.visualize(cs1, debug_grid=True)

In [ ]:
modes1 = mw.compute_modes(cs=cs1, num_modes=6)
mw.visualize(modes1, fields=["Ey", "Hy"], plot_width=3)

## Current

In [ ]:
cmap1 = colors.LinearSegmentedColormap.from_list(
    name="cmap1", colors=["#ffffff", "#ff0000"]
)
cmap2 = colors.LinearSegmentedColormap.from_list(
    name="cmap2", colors=["#ffffff", "#00ff00"]
)
cmap3 = colors.LinearSegmentedColormap.from_list(
    name="cmap3", colors=["#ffffff", "#0000ff"]
)

In [ ]:
plt.pcolormesh(
    cs1.mesh.Xx + 0.5,
    cs1.mesh.Yx + 0.5,
    cs1.nx,
    cmap=cmap1,
    vmin=1.0,
    alpha=0.3,
)
plt.pcolormesh(
    cs1.mesh.Xy + 0.5,
    cs1.mesh.Yy + 0.5,
    cs1.ny,
    cmap=cmap2,
    vmin=1.0,
    alpha=0.3,
)
plt.pcolormesh(
    cs1.mesh.Xz + 0.5,
    cs1.mesh.Yz + 0.5,
    cs1.nz,
    cmap=cmap3,
    vmin=1.0,
    alpha=0.3,
)
plt.xlim(0, W)
plt.ylim(0, H)
plt.show()

## Desired

In [ ]:
dx = cs1.mesh.x[1:] - cs1.mesh.x[:-1]
dy = cs1.mesh.y[1:] - cs1.mesh.y[:-1]
x_full = np.stack([cs1.mesh.x[:-1], cs1.mesh.x[:-1] + dx / 2], 1).ravel()
y_full = np.stack([cs1.mesh.y[:-1], cs1.mesh.y[:-1] + dy / 2], 1).ravel()
Y_full, X_full = np.meshgrid(y_full, x_full)
n_full = np.ones_like(X_full)
n_full[(4 <= Y_full) & (Y_full <= 4 + hs)] = nsi
n_full[
    (4 <= Y_full)
    & (Y_full <= 4 + hc)
    & (W // 2 - w // 2 <= X_full)
    & (X_full <= W // 2 + w // 2)
] = nsi
# n_full[Y_full <= 4] = nsl

mz = np.zeros_like(n_full, dtype=bool)
mz[::2, ::2] = True
mx = np.zeros_like(n_full, dtype=bool)
mx[1::2, ::2] = True
my = np.zeros_like(n_full, dtype=bool)
my[::2, 1::2] = True
m_ = np.zeros_like(n_full, dtype=bool)
m_[1::2, 1::2] = True
plot_mask = n_full > 1
n_mask = (
    1.0 * (plot_mask & mx)
    + 2.0 * (plot_mask & my)
    + 3.0 * (plot_mask & mz)
    + 4.0 * (plot_mask & m_)
)
plt.pcolormesh(X_full, Y_full, n_mask, cmap="Blues")

x_ticks = np.sort(np.unique(X_full.ravel()))[::2]
y_ticks = np.sort(np.unique(Y_full.ravel()))[::2]
plt.xticks(x_ticks - 0.25 * dx, [f"" for x in x_ticks - 0.25 * dx])
plt.yticks(y_ticks - 0.25 * dy, [f"" for y in y_ticks - 0.25 * dy])
plt.xticks(x_ticks + 0.25 * dx, [f"" for x in x_ticks + 0.25 * dx], minor=True)
plt.yticks(y_ticks + 0.25 * dy, [f"" for y in y_ticks + 0.25 * dy], minor=True)
plt.grid(True, which="major", ls="-")
plt.grid(True, which="minor", ls=":")
plt.xlim(0, W)
plt.ylim(0, W)
plt.show()

In [ ]:
def get_boundary_mask_horizontal(n_full, negate=False):
    mask = np.zeros((n_full.shape[0] + 2, n_full.shape[1] + 2), dtype=bool)
    mask[1:-1, 1:-1] = n_full > 1
    if negate:
        mask = ~mask
    mask1 = (convolve(np.array(mask[:, :], dtype=int), np.array([[-1, 1]])) > 0)[:, :]
    mask2 = (convolve(np.array(mask[:, ::-1], dtype=int), np.array([[-1, 1]])) > 0)[
        :, ::-1
    ]
    mask3 = (convolve(np.array(mask[::-1, :], dtype=int), np.array([[-1, 1]])) > 0)[
        ::-1, :
    ]
    mask = (mask1 | mask2 | mask3)[1:-1, 1:-1]
    # don't highlight mask at edge of simulation area
    mask[:, 0] = mask[:, -1] = False
    return mask


def get_boundary_mask_vertical(n_full, negate=False):
    return get_boundary_mask_horizontal(n_full.T, negate=negate).T


def get_boundary_mask(n_full, negate=False):
    mask1 = get_boundary_mask_horizontal(n_full, negate=negate)
    mask2 = get_boundary_mask_vertical(n_full, negate=negate)
    return mask1 | mask2


mh = np.zeros_like(n_full, dtype=bool)
mh[:, ::2] = True

mv = np.zeros_like(n_full, dtype=bool)
mv[::2, :] = True

maskv = get_boundary_mask_vertical(n_full)
maskv = maskv & (~mv)

maskh = get_boundary_mask_horizontal(n_full)
maskh = maskh & (~mh)

mask = maskv | maskh

maskc1 = convolve(np.asarray(mask, dtype=float), np.array([[-1.0, +1.0], [+1.0, -1.0]])) > 1  # fmt: skip
maskc2 = convolve(np.asarray(mask, dtype=float), np.array([[0.0, 0.0], [+1.0, -1.0], [-1.0, +1.0]])) > 1  # fmt: skip
mask = mask | maskc1 | maskc2

plot_mask = (n_full > 1) & (~mask)
n_mask = (
    1.0 * (plot_mask & mx)
    + 2.0 * (plot_mask & my)
    + 3.0 * (plot_mask & mz)
    + 4.0 * (plot_mask & m_)
)


plt.pcolormesh(X_full, Y_full, n_mask, cmap="Blues", alpha=1.0)
plt.pcolormesh(X_full, Y_full, mask, cmap=cmap1, alpha=0.5)
x_ticks = np.sort(np.unique(X_full.ravel()))[::2]
y_ticks = np.sort(np.unique(Y_full.ravel()))[::2]
plt.xticks(x_ticks - 0.25 * dx, [f"" for x in x_ticks - 0.25 * dx])
plt.yticks(y_ticks - 0.25 * dy, [f"" for y in y_ticks - 0.25 * dy])
plt.xticks(x_ticks + 0.25 * dx, [f"" for x in x_ticks + 0.25 * dx], minor=True)
plt.yticks(y_ticks + 0.25 * dy, [f"" for y in y_ticks + 0.25 * dy], minor=True)
plt.grid(True, which="major", ls="-")
plt.grid(True, which="minor", ls=":")
plt.xlim(0, W)
plt.ylim(0, W)
plt.show()

In [ ]:
n_full[n_mask < 1] = 1.0

Xz = X_full[::2, ::2]
Yz = Y_full[::2, ::2]
nz = n_full[::2, ::2]

Xx = X_full[1::2, ::2]
Yx = Y_full[1::2, ::2]
nx = n_full[1::2, ::2]

Xy = X_full[::2, 1::2]
Yy = Y_full[::2, 1::2]
ny = n_full[::2, 1::2]

In [ ]:
plt.pcolormesh(Xx + 0.5, Yx + 0.5, nx, cmap=cmap1, vmin=1.0, alpha=0.3)
plt.pcolormesh(Xy + 0.5, Yy + 0.5, ny, cmap=cmap2, vmin=1.0, alpha=0.3)
plt.pcolormesh(Xz + 0.5, Yz + 0.5, nz, cmap=cmap3, vmin=1.0, alpha=0.3)
x_ticks = np.sort(np.unique(X_full.ravel()))[::2]
y_ticks = np.sort(np.unique(Y_full.ravel()))[::2]
plt.xticks(x_ticks - 0.25 * dx, [f"" for x in x_ticks - 0.25 * dx])
plt.yticks(y_ticks - 0.25 * dy, [f"" for y in y_ticks - 0.25 * dy])
plt.xticks(x_ticks + 0.25 * dx, [f"" for x in x_ticks + 0.25 * dx], minor=True)
plt.yticks(y_ticks + 0.25 * dy, [f"" for y in y_ticks + 0.25 * dy], minor=True)
plt.grid(True, which="major", ls="-")
plt.grid(True, which="minor", ls=":")
plt.xlim(0 + dx.mean(), W + dx.mean())
plt.ylim(0 + dy.mean(), W + dy.mean())
plt.show()

In [ ]:
def compute_modes(
    x,
    y,
    nx,
    ny,
    nz,
    num_modes=10,
    wl=1.5,
):
    od = np.zeros_like(nx)
    eps_cross = [nx**2, od, od, od, ny**2, od, od, od, nz**2]
    mode_spec = td.ModeSpec(
        num_modes=num_modes,
        num_pml=(0, 0),
    )
    ((Ex, Ey, Ez), (Hx, Hy, Hz)), neffs, _ = _compute_modes(
        eps_cross=eps_cross,
        coords=[x, y],
        freq=td.C_0 / wl,
        mode_spec=mode_spec,
        precision="double",
    )
    modes = [
        mw.Mode(
            cs=cs1,
            Ex=Ex.squeeze()[..., i],
            Ey=Ey.squeeze()[..., i],
            Ez=Ez.squeeze()[..., i],
            Hx=Hx.squeeze()[..., i],
            Hy=Hy.squeeze()[..., i],
            Hz=Hz.squeeze()[..., i],
            neff=neffs.squeeze()[i],
        )
        for i in range(num_modes)
    ]
    # modes = [zero_phase(normalize_energy(mode)) for mode in modes]
    # modes = sorted(modes, key=lambda m: float(np.real(m.neff)), reverse=True)
    return modes

In [ ]:
modes2 = compute_modes(
    x=cs1.mesh.x,
    y=cs1.mesh.y,
    nx=nx,
    ny=ny,
    nz=nz,
)
mw.visualize([modes2[i] for i in [0, 2]], fields=["Ey", "Hx"], plot_width=3)